<a href="https://colab.research.google.com/github/VladislavTokarev02/technical/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тестовое задание для кандидата в команду валидации Альфа-Банка

**Привет!** На связи команда валидации Альфа-Банка. Мы отвечаем за то, чтобы модели, которые используются для принятия решений, были надежными, справедливыми и понятными. Твое задание — провести моделирование и валидацию кредитной модели. Удачи, и помни: мы верим в твои силы! 💪  

---

## Цель  
Проверить навыки анализа данных, валидации ML-моделей и умение формулировать выводы для бизнеса.  

---

## Датасет  
**Название:** [Give Me Some Credit](https://www.kaggle.com/c/GiveMeSomeCredit/data)  
**Описание:**  
Прогнозирование дефолта заемщика на основе финансовых и демографических признаков.  

**Признаки:**  
- `RevolvingUtilizationOfUnsecuredLines` (использование кредитных линий),  
- `age`, `MonthlyIncome`, `NumberOfDependents` и др.  

**Целевая переменная:** `SeriousDlqin2yrs` (дефолт: 0/1).  

---

## Задача  
Проведите валидацию модели кредитного скоринга (логистическая регрессия или градиентный бустинг) и подготовьте отчет.  

---

## Шаги  

### 1. Анализ и предобработка данных  
- Обработайте пропуски (например, в `MonthlyIncome`).  
- Исследуйте выбросы (например, возраст < 18 лет).  
- Визуализируйте распределения ключевых признаков.  
- Предложите методы борьбы с дисбалансом классов.  

### 2. Построение и оценка модели  
- Разделите данные на train/validation/test.  
- Обучите модель (логистическая регрессия или CatBoost/XGBoost).  
- Рассчитайте метрики: **AUC-ROC, Precision, Recall, F1-Score**.  
- Проверьте устойчивость модели через кросс-валидацию (5 folds).  

### 3. Интерпретация и этика  
- Выделите **топ-5 признаков**, влияющих на прогноз (SHAP/LIME).  
- Проверьте логичность влияния признаков (например, высокая долговая нагрузка → выше риск дефолта).  
- Оцените fairness модели: сравните метрики (FPR, TPR) для групп (например, **молодые** vs **старше 40 лет**).  

### 4. Отчет  
Подготовьте общие выводы по проведенной валидации, включив:  
- Выводы о качестве модели и её ограничениях.  
- Рекомендации по улучшению (например, сбор дополнительных данных).  
- Пример: *«Как изменится прогноз, если у заемщика появится иждивенец?»*  

---

## Технические требования  
- Язык: **Python** (Jupyter Notebook).  
- Код должен быть читаемым и содержать комментарии.  

---

## Критерии оценки  
1. Глубина анализа данных и обработки выбросов.  
2. Корректность выбранных метрик и их интерпретация.  
3. Качество визуализаций (распределения, важность признаков).  
4. Практичность рекомендаций в отчете.    

---

**Срок выполнения:** 7 дней.  
**Формат сдачи:**  
- Ноутбук в Collab/GitHub-репозиторий.    

---

Это задание покажет, как вы подходите к анализу реальных данных и делаете ML-модели прозрачными для бизнеса. Ждем твою работу! 🚀  

In [31]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split

random.seed(42)


pd.set_option('display.max_columns', None)

In [32]:
data_dict = pd.read_excel('Data Dictionary.xls')

Интерпретация столбцов датасета из data_dict

| Имя переменной                      | Описание                                                                                                                                         |
|-----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| SeriousDlqin2yrs                    | Лицо имело задолженность 90+ дней или более за последние 2 года.                                                                                      |
| RevolvingUtilizationOfUnsecuredLines| Общий баланс по кредитным картам и персональным кредитным линиям (без недвижимости и займов на покупку авто) делённый на сумму кредитных лимитов.     |
| age                                 | Возраст заёмщика в годах.                                                                                                                            |
| NumberOfTime30-59DaysPastDueNotWorse| Количество случаев, когда заёмщик был в просрочке 30–59 дней, но не более, за последние 2 года.                                                      |
| DebtRatio                           | Соотношение ежемесячных платежей по долгам (алименты, расходы на жизнь и т. д.) к ежемесячному валовому доходу.                                       |
| MonthlyIncome                       | Ежемесячный доход.                                                                                                                                   |
| NumberOfOpenCreditLinesAndLoans     | Количество открытых кредитов (автокредит или ипотека) и кредитных линий (например, кредитные карты).                                                 |
| NumberOfTimes90DaysLate             | Количество случаев, когда заёмщик был в просрочке на 90 дней или более.                                                                              |
| NumberRealEstateLoansOrLines        | Количество займов под недвижимость, включая кредитные линии на недвижимость.                                                                         |
| NumberOfTime60-89DaysPastDueNotWorse| Количество случаев, когда заёмщик был в просрочке 60–89 дней, но не более, за последние 2 года.                                                      |
| NumberOfDependents                  | Количество иждивенцев в семье (исключая самого заёмщика, супругу/супруга и т. д.).                                                                    |

In [33]:
train = pd.read_csv('cs-training.csv')

In [34]:
train.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [35]:
df_test = pd.read_csv('cs-test.csv')

In [36]:
df_test.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [37]:
train.shape, df_test.shape

((150000, 12), (101503, 12))

In [39]:
train.isna().mean().round(4).to_frame().sort_values(by=0, ascending = False).style.format('{:.2%}').background_gradient('coolwarm')

,0
MonthlyIncome,19.82%
NumberOfDependents,2.62%
Unnamed: 0,0.00%
SeriousDlqin2yrs,0.00%
RevolvingUtilizationOfUnsecuredLines,0.00%
age,0.00%
NumberOfTime30-59DaysPastDueNotWorse,0.00%
DebtRatio,0.00%
NumberOfOpenCreditLinesAndLoans,0.00%
NumberOfTimes90DaysLate,0.00%


In [40]:
test.isna().mean().round(4).to_frame().sort_values(by=0, ascending = False).style.format('{:.2%}').background_gradient('coolwarm')

,0
SeriousDlqin2yrs,100.00%
MonthlyIncome,19.81%
NumberOfDependents,2.59%
Unnamed: 0,0.00%
RevolvingUtilizationOfUnsecuredLines,0.00%
age,0.00%
NumberOfTime30-59DaysPastDueNotWorse,0.00%
DebtRatio,0.00%
NumberOfOpenCreditLinesAndLoans,0.00%
NumberOfTimes90DaysLate,0.00%


train

Как мы видим - пропуски есть только в 2 столбцах - ежемесячный доход и количество иждивенцев в семье.

- Количество иждивенцев в семье: доля пропусков немного выше 2.5%, таким образом, можно просто удалить эти пропуски без заполнения.
- Ежемесячный доход: в простейшем случае можно заполнить медианой, хотя существуют вариации заполнения с учётом k-ближайших соседей, линейной регрессии, k-means.

In [46]:
train['MonthlyIncome'] = train['MonthlyIncome'].fillna(train['MonthlyIncome'].median())
test['MonthlyIncome'] = test['MonthlyIncome'].fillna(train['MonthlyIncome'].median())

train['NumberOfDependents'] = train['NumberOfDependents'].dropna()
test['NumberOfDependents'] = test['NumberOfDependents'].dropna()

In [47]:
train.isna().sum(), test.isna().sum()

(Unnamed: 0                                 0
 SeriousDlqin2yrs                           0
 RevolvingUtilizationOfUnsecuredLines       0
 age                                        0
 NumberOfTime30-59DaysPastDueNotWorse       0
 DebtRatio                                  0
 MonthlyIncome                              0
 NumberOfOpenCreditLinesAndLoans            0
 NumberOfTimes90DaysLate                    0
 NumberRealEstateLoansOrLines               0
 NumberOfTime60-89DaysPastDueNotWorse       0
 NumberOfDependents                      3924
 dtype: int64,
 Unnamed: 0                                   0
 SeriousDlqin2yrs                        101503
 RevolvingUtilizationOfUnsecuredLines         0
 age                                          0
 NumberOfTime30-59DaysPastDueNotWorse         0
 DebtRatio                                    0
 MonthlyIncome                                0
 NumberOfOpenCreditLinesAndLoans              0
 NumberOfTimes90DaysLate                      0
 

Разобьём cs-training.csv на тренировочную и валидационную выборки.

In [28]:
df_train, df_val = train_test_split(train, test_size = 0.2, random_state = 42)

In [30]:
print("Размер тренировочного набора:", df_train.shape)
print("Размер валидационного набора:", df_val.shape)
print("Размер тестового набора:", test.shape)

Размер тренировочного набора: (120000, 12)
Размер валидационного набора: (30000, 12)
Размер тестового набора: (101503, 12)
